<a href="https://colab.research.google.com/github/Aynazf/ABSA-PyTorch/blob/master/Pass_K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install evaluate
!pip install xlsxwriter

In [ ]:
import os
os.environ["HF_ALLOW_CODE_EVAL"] = "1"

In [ ]:
from evaluate import load
# code_eval = load("code_eval")
code_eval_metric = load("code_eval")

In [ ]:
import textwrap
def proccess_test_cases(test_case):

    test_case=test_case.replace('def check(candidate):', '')
    test_case=textwrap.dedent(test_case)
    test_case=test_case.replace('assert candidate', 'assert calculate_risk_weight')

    return test_case

In [ ]:
import pandas as pd
test_cases=pd.read_excel('/content/drive/MyDrive/RegAI/paper/code_dataset.xlsx',sheet_name=None)
keys=list(test_cases.keys())

valid_cases=[]
invalid_cases=[]
both_cases=[]
for key in keys:
    sheet=test_cases[key]
    test_valid=sheet['Test_Valid'][0]
    test_valid=proccess_test_cases(test_valid)
    valid_cases.append(test_valid)
    test_invalid=sheet['Test_Invalid'][0]
    test_invalid=proccess_test_cases(test_invalid)
    invalid_cases.append(test_invalid)
    both_cases.append(test_valid+test_invalid)

In [ ]:
temp=0
df=pd.read_excel(f'/content/drive/MyDrive/RegAI/paper/Mistral_large/functions_5000_T{temp}.xlsx')
keys=list(df.keys())

predictions=[]
for key in keys:
    curr_sheet=[]
    for item in df[key]:
      curr_sheet.append(item)
    predictions.append(curr_sheet)


In [ ]:
pass_at_k, results_valid = code_eval_metric.compute(references=valid_cases, predictions=predictions, k=[1, 5, 10])
pass_at_k_invalid, results_invalid = code_eval_metric.compute(references=invalid_cases, predictions=predictions, k=[1, 5, 10])
pass_at_k_both, results_both = code_eval_metric.compute(references=both_cases, predictions=predictions, k=[1, 5, 10])
print("Pass valid",pass_at_k)
print("Pass invalid" , pass_at_k_invalid)
print("Pass both" , pass_at_k_both)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Pass valid {'pass@1': 0.1846153846153846, 'pass@5': 0.22893772893772896, 'pass@10': 0.23076923076923078}
Pass invalid {'pass@1': 0.5999999999999999, 'pass@5': 0.6904761904761905, 'pass@10': 0.6923076923076923}
Pass both {'pass@1': 0.15384615384615385, 'pass@5': 0.15384615384615385, 'pass@10': 0.15384615384615385}


In [ ]:
# Specify the Excel file path
excel_file = f'Mistral_Large_temp_5000_{temp}_results.xlsx'

# Create a Pandas Excel writer using XlsxWriter as the engine
with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
    # Loop to create 13 sheets with different DataFrames
    for i in range(1, 14):
        passes=[]
        results=[]
        types=[]
        for item in results_valid:
            for id in results_valid[item]:
              if id[1]['task_id']==i-1:
                  types.append('valid')
                  passes.append(id[1]['passed'])
                  results.append(id[1]['result'])
        passes.append(' ')
        results.append(' ')
        types.append(' ')
        for item in results_invalid:
            for id in results_invalid[item]:
              if id[1]['task_id']==i-1:
                  types.append('invalid')
                  passes.append(id[1]['passed'])
                  results.append(id[1]['result'])
        passes.append(' ')
        results.append(' ')
        types.append(' ')
        for item in results_both:
            for id in results_both[item]:
              if id[1]['task_id']==i-1:
                  types.append('invalid')
                  passes.append(id[1]['passed'])
                  results.append(id[1]['result'])
        # Generate different data for each sheet
        data = {
            'type': types,
            'passed': passes,
            'result': results
        }
        df = pd.DataFrame(data)  # Create a DataFrame with different data for each sheet
        sheet_name = f'Sheet{i}'  # Name the sheet as 'Sheet1', 'Sheet2', ..., 'Sheet13'
        df.to_excel(writer, sheet_name=sheet_name, index=False)

# The Excel file is now saved with 13 sheets with different DataFrames
print(f'Excel file saved as {excel_file}')

Excel file saved as Mistral_Large_temp_5000_0_results.xlsx


In [ ]:
results